In [11]:
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import pandas as pd
import gurobipy as gp
import numpy as np
from gurobipy import Model, GRB, quicksum

# Load passenger data
passenger_data = pd.read_csv('testing.csv')

# Extract coordinates and IDs
n = len(passenger_data)
xc = passenger_data['Latitude'].values
yc = passenger_data['Longitude'].values
ids = passenger_data['PassengerID'].values

# Define depot coordinates (you can adjust this to a central location)
depot_latitude = 42.3527  # Example: Linden Street
depot_longitude = -71.1280
xc = np.append([depot_latitude], xc)
yc = np.append([depot_longitude], yc)
# Define vehicle capacities
vehicle_capacities = [14] * 7 + [4]
num_vehicles = len(vehicle_capacities)
Q = max(vehicle_capacities)  # Maximum capacity (used in constraints)
# Vehicle and client sets
N = [i for i in range(1, n+1)]  # Client IDs
V = [0] + N                    # Include the depot
A = [(i, j) for i in V for j in V if i != j]  # All arcs
# Costs (distances between nodes)
c = {(i, j): np.hypot(xc[i] - xc[j], yc[i] - yc[j]) for i, j in A}

# Demand for each passenger
q = {i: 1 for i in N}
# Optimization Model
mdl = Model('CVRP')
x = mdl.addVars(A, vtype=GRB.BINARY)
u = mdl.addVars(N, vtype=GRB.CONTINUOUS)  # Load of each client
mdl.modelSense = GRB.MINIMIZE
mdl.setObjective(quicksum(x[a] * c[a] for a in A))
# Constraints
mdl.addConstrs(quicksum(x[i, j] for j in V if j != i) == 1 for i in N)
mdl.addConstrs(quicksum(x[j, i] for j in V if j != i) == 1 for i in N)
mdl.addConstrs((x[i, j] == 1) >> (u[i] + q[i] == u[j]) for i, j in A if i != 0 and j != 0)
mdl.addConstrs(u[i] >= q[i] for i in N)
mdl.addConstrs(u[i] <= Q for i in N)
mdl.addConstr(quicksum(q[i] for i in N) <= sum(vehicle_capacities))





<gurobi.Constr *Awaiting Model Update*>

In [12]:
# Ad Hoc Request Function
def handle_ad_hoc_request(ad_hoc_requests, passenger_data):
    """
    Updates the passenger data for ad hoc requests by replacing their residential locations
    with their requested destinations.
    :param ad_hoc_requests: List of tuples [(PassengerID, latitude, longitude), ...]
    :param passenger_data: Existing DataFrame of passengers.
    :return: Updated passenger data.
    """
    global xc, yc, ids, n
    
    for request in ad_hoc_requests:
        passenger_id, latitude, longitude = request
        
        # Check if the PassengerID exists in the data
        if passenger_id in passenger_data['PassengerID'].values:
            # Update the passenger's destination
            passenger_data.loc[passenger_data['PassengerID'] == passenger_id, ['Latitude', 'Longitude']] = latitude, longitude
        else:
            print(f"PassengerID {passenger_id} not found!")
# Update coordinates and IDs
xc = np.append([depot_latitude], passenger_data['Latitude'].values)
yc = np.append([depot_longitude], passenger_data['Longitude'].values)
ids = passenger_data['PassengerID'].values
n = len(passenger_data) - 1  # Exclude depot


In [ ]:
from folium.plugins import MarkerCluster
from folium import DivIcon
import random

def visualize_routes(active_arcs, car_assignments, xc, yc, ids, depot_latitude, depot_longitude, passenger_data):
    """
    Visualize the optimized routes using Folium.
    :param active_arcs: List of active arcs (routes) from the optimization.
    :param xc: Array of x-coordinates (latitudes).
    :param yc: Array of y-coordinates (longitudes).
    :param ids: List of passenger IDs.
    :param depot_latitude: Latitude of the depot.
    :param depot_longitude: Longitude of the depot.
    """
    
    # Create a map centered around the depot
    route_map = folium.Map(location=[depot_latitude, depot_longitude], zoom_start=13)


    # Add the depot marker
    folium.Marker(
        location=[depot_latitude, depot_longitude],
        popup="Depot",
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(route_map)

    # Group arcs by car
    car_to_arcs = {}
    for idx, arc in enumerate(active_arcs):
        car_id = car_assignments[idx]
        if car_id not in car_to_arcs:
            car_to_arcs[car_id] = []
        car_to_arcs[car_id].append(arc)
        
    car_colors = {car_id: f"#{''.join(random.choices('0123456789ABCDEF', k=6))}" for car_id in car_to_arcs.keys()}
    # Add a cluster for passenger markers
    marker_cluster = MarkerCluster().add_to(route_map)

    # Group passengers by location
    location_groups = {}
    for i in range(1, len(ids)):  # Skip the depot (index 0)
        loc = (xc[i], yc[i])
        if loc not in location_groups:
            location_groups[loc] = []
        location_groups[loc].append(ids[i])

    # Add markers for each unique location with passenger count on the icon
    for loc, passengers in location_groups.items():
        passenger_count = len(passengers)
        popup_content = f"Passengers: {', '.join(map(str, passengers))}"

        # Custom icon to show the passenger count
        folium.Marker(
            location=loc,
            popup=popup_content,
            icon=DivIcon(
                icon_size=(20, 20),
                icon_anchor=(10, 10),
                html=f'<div style="font-size: 12px; color: white; background: blue; '
                     f'border-radius: 50%; text-align: center; width: 24px; height: 24px; line-height: 24px;">'
                     f'{passenger_count}</div>'
            )
        ).add_to(route_map)

    # Draw routes
    for car_id, arcs in car_to_arcs.items():
        color = car_colors[car_id]  # Cycle through colors if cars > 7
        for i, j in arcs:
            folium.PolyLine(
            locations=[[xc[i], yc[i]], [xc[j], yc[j]]],
            color=color,
            weight=2.5,
            popup=f"Car {car_id + 1}"
        ).add_to(route_map)
        
    
    # Display the map
    route_map.save('optimized_routes.html')
    print("Map saved as 'optimized_routes.html'. Open this file in your browser to view the routes.")
   

In [15]:
passenger_data = pd.read_csv('testing.csv')
#CLI Interface
def main():
    global passenger_data 
    print("Welcome to the Shuttle Scheduling System")
    while True:
        print("\n1. Optimize Schedule")
        print("2. Add Ad Hoc Request")
        print("3. Exit")
        
        
        try:
            choice = int(input("Choose an option: "))
            if choice not in [1, 2, 3]:
                raise ValueError
        except ValueError:
            print("Invalid choice. Please enter 1, 2, or 3.")
            continue
        

        if choice == 1:
            print("Starting optimization...")
            mdl.Params.TimeLimit = 20  # Limit optimization to 10 seconds
            mdl.Params.MIPGap = 0.3   # Allow solutions within 20% of optimal
            mdl.Params.OutputFlag = 1  # Enable solver logs
            mdl.optimize()
            print("Optimization complete.")


            # Extract active arcs
            active_arcs = [a for a in A if x[a].x > 0.99]  # Extract active arcs
            print(f"Total optimized distance: {mdl.ObjVal}")

            car_assignments = [i % 7 for i in range(len(active_arcs))]
            # Visualize the routes
            visualize_routes(active_arcs, car_assignments, xc, yc, ids, depot_latitude, depot_longitude, passenger_data)
      

        elif choice == 2:
            try:
                passenger_id = input("Enter PassengerID: ")
                latitude = float(input("Enter new destination latitude: "))
                longitude = float(input("Enter new destination longitude: "))
                passenger_data = handle_ad_hoc_request([(passenger_id, latitude, longitude)], passenger_data)
                print(f"Ad hoc request for {passenger_id} processed successfully!")
            except ValueError:
                print("Invalid input. Please enter a valid PassengerID, latitude, and longitude.")
                continue

            
            print(f"Ad hoc request for {passenger_id} processed successfully!")
            
        elif choice == 3:
            print("Exiting the system. Goodbye!")
            break

        else:
            print("Invalid choice. Please try again.")



main()


Welcome to the Shuttle Scheduling System

1. Optimize Schedule
2. Add Ad Hoc Request
3. Exit
Invalid input. Please enter a valid PassengerID, latitude, and longitude.

1. Optimize Schedule
2. Add Ad Hoc Request
3. Exit
Starting optimization...
Set parameter TimeLimit to value 20
Set parameter MIPGap to value 0.3
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[rosetta2] - Darwin 23.6.0 23G93)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  20
MIPGap  0.3

Optimize a model with 333 rows, 7055 columns and 13944 nonzeros
Model fingerprint: 0x64221448
Model has 6806 simple general constraints
  6806 INDICATOR
Variable types: 83 continuous, 6972 integer (6972 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
  GenCon rhs range [1e+00, 1e+00]
  GenCo